<a href="https://colab.research.google.com/github/ebamberg/TechPackersColab/blob/main/examples_LLM_fine_tuning_using_SFT_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

see: https://huggingface.co/docs/trl/main/en/sft_trainer

In [1]:
!pip install torch --quiet
!pip install transformers --upgrade --quiet
!pip install datasets --quiet
# Attention: we have to install peft before trl
!pip install peft
!pip install -U bitsandbytes
!pip install accelerate
!pip install trl --quiet
# unsloth dependencies
#!pip install unsloth[colab-new]
#!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [2]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig,get_peft_model, TaskType
from transformers import BitsAndBytesConfig, TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch
# from unsloth import FastLanguageModel


In [3]:
## load first 100 rows if the train features of the dataset
dataset = load_dataset("imdb", split="train[:100]")


print (type(dataset))
print (dataset)
print (dataset.features)
print (dataset.column_names)
print (dataset["text"][:3])

# an alternative way to limit the dataset to the first 100 rows
#dataset_smaller = dataset.select(range(100))
#print (dataset_smaller)

<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['text', 'label'],
    num_rows: 100
})
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None)}
['text', 'label']
['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with

In [4]:
llmmodels =["facebook/opt-350m","EleutherAI/gpt-neo-125m","meta-llama/Meta-Llama-3.1-8B-Instruct","unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"]

modelid=llmmodels[0]

In [10]:
sft_config = SFTConfig(
    dataset_text_field="text",
    max_seq_length=1024,
    output_dir="/tmp",
     num_train_epochs=3,
     per_device_train_batch_size=16,
     gradient_accumulation_steps=2, # simulate larger batch sizes
    gradient_checkpointing=True, # gradient checkpointing saves even more memory
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    packing=True, # pack samples together for efficient training
 #   model_init_kwargs={
 #       "torch_dtype": "bfloat16",
 #   },
)

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16

)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

lora_config = LoraConfig(
    r=16,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=32,
    lora_dropout=0.05
)



base_model = AutoModelForCausalLM.from_pretrained(
    modelid,
    use_cache=False,  # False if gradient_checkpointing else True
    quantization_config=quantization_config,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(modelid)

lora_model = get_peft_model(base_model, lora_config)

lora_model.enable_input_require_grads()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
for name, param in lora_model.named_parameters():
    print(name, param.shape)

base_model.model.model.decoder.embed_tokens.weight torch.Size([50272, 512])
base_model.model.model.decoder.embed_positions.weight torch.Size([2050, 1024])
base_model.model.model.decoder.project_out.weight torch.Size([262144, 1])
base_model.model.model.decoder.project_in.weight torch.Size([262144, 1])
base_model.model.model.decoder.layers.0.self_attn.k_proj.weight torch.Size([524288, 1])
base_model.model.model.decoder.layers.0.self_attn.k_proj.bias torch.Size([1024])
base_model.model.model.decoder.layers.0.self_attn.v_proj.base_layer.weight torch.Size([524288, 1])
base_model.model.model.decoder.layers.0.self_attn.v_proj.base_layer.bias torch.Size([1024])
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_A.default.weight torch.Size([16, 1024])
base_model.model.model.decoder.layers.0.self_attn.v_proj.lora_B.default.weight torch.Size([1024, 16])
base_model.model.model.decoder.layers.0.self_attn.q_proj.base_layer.weight torch.Size([524288, 1])
base_model.model.model.decoder.laye

In [12]:
def prompt_generator(row):
  return ("Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{text}\n\n### Response:\n{label}").format_map(row)


trainer = SFTTrainer(
    lora_model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=sft_config,
  #  peft_config=peft_config,
    formatting_func=prompt_generator
)
trainstats=trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


In [13]:
print (trainstats)

TrainOutput(global_step=3, training_loss=3.7444661458333335, metrics={'train_runtime': 69.0281, 'train_samples_per_second': 1.434, 'train_steps_per_second': 0.043, 'total_flos': 123650665611264.0, 'train_loss': 3.7444661458333335, 'epoch': 2.0})


Example prediction

In [ ]:
#from transformers import pipeline
#generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
#generator("What can you tell me about I AM CURIOUS-YELLOW ?")